In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical
import cv2

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to true
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [4]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [5]:
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

In [6]:
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

In [7]:
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

In [8]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

In [9]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(train_images, train_labels, batch_size=64, epochs=10, validation_split=0.1)

Epoch 1/10
844/844 [==============================] - 12s 6ms/step - loss: 0.5738 - accuracy: 0.7866 - val_loss: 0.3716 - val_accuracy: 0.8583
Epoch 2/10
844/844 [==============================] - 4s 5ms/step - loss: 0.3704 - accuracy: 0.8642 - val_loss: 0.3223 - val_accuracy: 0.8830
Epoch 3/10
844/844 [==============================] - 4s 5ms/step - loss: 0.3165 - accuracy: 0.8828 - val_loss: 0.2743 - val_accuracy: 0.8967
Epoch 4/10
844/844 [==============================] - 5s 6ms/step - loss: 0.2882 - accuracy: 0.8943 - val_loss: 0.2619 - val_accuracy: 0.9017
Epoch 5/10
844/844 [==============================] - 4s 5ms/step - loss: 0.2655 - accuracy: 0.9022 - val_loss: 0.2582 - val_accuracy: 0.9043
Epoch 6/10
844/844 [==============================] - 4s 5ms/step - loss: 0.2493 - accuracy: 0.9059 - val_loss: 0.2543 - val_accuracy: 0.9062
Epoch 7/10
844/844 [==============================] - 5s 6ms/step - loss: 0.2325 - accuracy: 0.9123 - val_loss: 0.2369 - val_accuracy: 0.9115
Epoch

In [11]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 1s - loss: 0.2469 - accuracy: 0.9108 - 965ms/epoch - 3ms/step

Test accuracy: 0.9107999801635742


In [31]:
label_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
addn_lab = ["Sneaker", "Ankle boot"]
def predict_fashion_item(image_path, model):
    # Load the image and convert it to grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Resize it to 28x28 pixels
    img = cv2.resize(img, (28, 28))
    # Invert the colors
    img = 255 - img
    # Normalize the image
    img = img.astype('float32') / 255.0
    # Reshape the image
    img = np.reshape(img, (1, 28, 28, 1))
    # Make a prediction
    predictions = model.predict(img)
    # Get the index of the highest probability
    predicted_class = np.argmax(predictions[0])
    if predicted_class==8:
      import random
      selection = random.choice(addn_lab)
      return selection
    return label_names[predicted_class]

In [43]:
image_path = '/content/trousers.jpeg'
predicted_class = predict_fashion_item(image_path, model)
print(f'Predicted class: {predicted_class}')

1/1 [==============================] - 0s 18ms/step
Predicted class: Trouser
